In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import rioxarray 
import os

In [2]:
crops = ['Corn','Soybean', "Wheat", "Rice"]
phases = [2]

In [3]:
# Read in the data
gdf_dict = {}
for crop in crops:
    for phase in phases:
        rds = rioxarray.open_rasterio(
            "../" + os.sep + "processed" + os.sep + "output" + os.sep + crop + "_phase_" + str(phase) + "_rc_mean.asc"
        )
        rds = rds.squeeze().drop("spatial_ref").drop("band")
        rds.name = "data"
        df = rds.to_dataframe().reset_index().replace(-9, np.nan).dropna()
        # Convert to geodataframe
        gdf_dict[crop+str(phase)] = df

/tmp/ipykernel_229467/909564631.py:8: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  rds = rds.squeeze().drop("spatial_ref").drop("band")
/tmp/ipykernel_229467/909564631.py:8: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  rds = rds.squeeze().drop("spatial_ref").drop("band")
/tmp/ipykernel_229467/909564631.py:8: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  rds = rds.squeeze().drop("spatial_ref").drop("band")
/tmp/ipykernel_229467/909564631.py:8: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  rds = rds.squeeze().drop("spatial_ref").drop("band")


In [4]:
def dataframe_to_geopandas(df):
    # Convert the x and y columns to longitude and latitude, respectively
    geometry = [Point(x, y) for x, y in zip(df['x'], df['y'])]
    
    # Create a GeoDataFrame from the geometry and data columns
    gdf = gpd.GeoDataFrame(df['data'], geometry=geometry)

    # Convert to winkel tripel projection
    gdf.crs = "EPSG:4326"
    gdf = gdf.to_crs("+proj=wintri")

    return gdf

In [5]:
# Convert the dataframes to geodataframes
gdf_dict_gpd = {}
for i in gdf_dict.keys():
    gdf_dict_gpd[i] = dataframe_to_geopandas(gdf_dict[i])


In [6]:
# Read in the country borders
countries = gpd.read_file("../" + os.sep + "spatial_data" + os. sep +"countries" + os.sep + "World_Countries__Generalized_.shp")
countries.crs = "EPSG:4326"
countries = countries.to_crs('+proj=wintri')

In [19]:
# Iterate over the geodataframes and clip them to the country borders 
# And calculate the mean value of the data within the country borders
# Save mean values for the countries to a new dataframe and save it to a csv file
for crop in crops:
    for phase in phases:
        crop_gdf = gdf_dict_gpd[crop+str(phase)]
        countries_with_crops = gpd.sjoin(countries, crop_gdf, how="inner", op='intersects')
        for country in set(countries_with_crops['COUNTRY']):
            country_gdf = countries_with_crops[countries_with_crops['COUNTRY'] == country]
            mean_value = country_gdf['data'].mean()
            print(country, mean_value)
            with open("../" + os.sep + "processed" + os.sep + "output" + os.sep + "country_means" + os.sep + crop + str(phase) + "mean_values.csv", "a") as f:
                f.write(country + "," + str(mean_value) + "\n")



/home/florian/local/anaconda3/envs/pytradeshifts/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Morocco -0.014873775
Paraguay -0.3457571
Côte d'Ivoire -0.0535618
Czech Republic -0.44120315
Egypt -0.66834205
Iran -0.50619406
Afghanistan -0.0069841375
Trinidad and Tobago -0.2804461
Namibia 0.0
Chile -0.607358
Cuba -0.01653936
Uruguay -0.063974485
Gabon -0.015357812
North Korea -0.03156154
Togo -0.0134423515
Albania -0.2564511
Colombia -0.17090099
Hungary -0.39291513
Romania -0.1565478
Saudi Arabia -0.30243745
Austria -0.58547324
South Africa -0.4220563
Guinea-Bissau 0.0
Kenya -0.04290999
Slovakia -0.33804297
Myanmar -0.4252847
Honduras -0.05603922
Nicaragua -0.010549765
Laos -0.5154352
Croatia -0.30469182
Belarus -0.3309698
Netherlands -0.6116485
Kazakhstan -0.26559198
Bosnia and Herzegovina -0.10748237
Tanzania -0.045169134
Italy -0.45113698
Thailand -0.16381656
Ethiopia -0.21410877
Costa Rica -0.27426535
Canada -0.4858136
Germany -0.5045182
Indonesia -0.32243893
Philippines -0.13481711
Tajikistan -0.63405085
Switzerland -0.52874625
Peru -0.24581076
Somalia -0.054801896
Turkmenist

/home/florian/local/anaconda3/envs/pytradeshifts/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Nigeria -0.0013932579
Uganda -0.0034866834
Russian Federation -0.18152225
Paraguay -0.34495658
Zambia -0.26496938
Japan -0.043614764
Serbia -0.43032175
Ecuador -0.09682868
Bosnia and Herzegovina -0.31742883
Tanzania 0.0
Iran -0.39566317
Mexico -0.026399884
India -0.12350551
Italy -0.47046778
Thailand -0.15652375
Canada -0.38444188
Indonesia -0.035777677
Turkey -0.23946588
Moldova -0.11012391
Uruguay -0.23938805
South Korea -0.09982125
United States -0.381662
North Korea -0.024885166
Albania -0.12906206
Colombia -0.14637557
Hungary -0.45566922
Romania -0.27016646
Vietnam -0.10186481
Brazil -0.4273134
France -0.4510687
Nepal -0.12324459
Rwanda -0.009587429
Slovenia -0.4757658
Austria -0.46825817
South Africa -0.19130789
Benin 0.0
China -0.16270654
Slovakia -0.2009895
Myanmar -0.08635248
Ukraine -0.24332032
Zimbabwe -0.06335761
Laos -0.0062974147
Bolivia -0.044381756
Croatia -0.37113982
Argentina -0.3171666
Cambodia -0.029056469


/home/florian/local/anaconda3/envs/pytradeshifts/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Morocco -0.23370865
Paraguay -0.17195873
Czech Republic -0.5529134
Egypt -0.67366916
Iran -0.21129471
Afghanistan -0.26180238
Chile -0.63719183
Uruguay -0.46621126
North Korea -0.004113271
Albania -0.5471491
Colombia -0.2149043
Hungary -0.5224952
Romania -0.3889447
Saudi Arabia -0.70882076
Austria -0.60726374
South Africa -0.56306744
Kenya -0.25715074
Armenia -0.31690934
Luxembourg -0.5937422
Slovakia -0.38601017
Myanmar -0.11044377
Croatia -0.5014653
Belarus -0.33537
Netherlands -0.71882415
Denmark -0.6643078
Kazakhstan -0.034888174
Tunisia -0.28201833
Japan -0.1657703
Bosnia and Herzegovina -0.364229
Latvia -0.3545334
Tanzania -0.029894684
Libya -0.00030406142
Italy -0.3318474
Ethiopia -0.20295304
Canada -0.43329933
Germany -0.68041223
Tajikistan -0.4272625
Switzerland -0.5605626
Peru -0.10628239
Turkmenistan -0.18316974
Spain -0.4116935
Estonia -0.26945812
Nepal -0.24644074
Jordan 0.0
Sudan -0.060610592
Ukraine -0.4261557
Bolivia -0.07564715
Argentina -0.29416096
Poland -0.5124472
L

/home/florian/local/anaconda3/envs/pytradeshifts/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Morocco -0.08597795
Paraguay -0.48123965
Côte d'Ivoire -0.04549557
Egypt -0.6691531
Iran -0.1826862
Afghanistan -0.054813776
Trinidad and Tobago 0.0
Chile -0.17227526
Cuba -0.06713452
Uruguay -0.45826718
North Korea -0.050203342
Togo -0.19530456
Colombia -0.15817761
French Guiana -0.10585835
Guinea-Bissau -0.10776577
Kenya -0.1800231
Myanmar -0.2534198
Honduras -0.51276433
Liberia 0.0
Nicaragua -0.43958065
Laos -0.22879888
Japan -0.1943894
Tanzania -0.0995686
Italy -0.3540604
Thailand -0.1681402
Costa Rica -0.26522875
Indonesia -0.20718735
Philippines -0.099893674
Tajikistan -0.18524742
Peru -0.46313393
Turkmenistan 0.0
Panama -0.029805824
Spain -0.4393596
Nepal -0.116872326
El Salvador -0.66608745
Mozambique -0.0036804664
Guinea -0.0027947417
Bolivia -0.020117583
Madagascar -0.1447075
Argentina -0.49140403
Cambodia -0.13421892
Nigeria -0.032368366
Uganda -0.0707406
Congo DRC 0.0
Senegal -0.30406475
Chad 0.0
Mali -0.07265715
Mauritania -0.3688561
Mexico -0.5047563
Gambia -0.043672387
A

In [11]:
# Do a spatial join between the country borders and the crop data
countries_with_crops = gpd.sjoin(countries, crop_gdf, how="inner", op='intersects')
countries_with_crops.head()

/home/florian/local/anaconda3/envs/pytradeshifts/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,SHAPE_Leng,SHAPE_Area,geometry,index_right,data
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3975505,-0.128534
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3971185,-0.627760
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3979824,-0.508288
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3979823,-0.468903
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3979822,-0.410291


In [16]:
for country in set(countries_with_crops['COUNTRY']):
    print(country)
    print(countries_with_crops[countries_with_crops['COUNTRY'] == country]['data'].mean())


Morocco
-0.014873775
Paraguay
-0.3457571
Côte d'Ivoire
-0.0535618
Czech Republic
-0.44120315
Egypt
-0.66834205
Iran
-0.50619406
Afghanistan
-0.0069841375
Trinidad and Tobago
-0.2804461
Namibia
0.0
Chile
-0.607358
Cuba
-0.01653936
Uruguay
-0.063974485
Gabon
-0.015357812
North Korea
-0.03156154
Togo
-0.0134423515
Albania
-0.2564511
Colombia
-0.17090099
Hungary
-0.39291513
Romania
-0.1565478
Saudi Arabia
-0.30243745
Austria
-0.58547324
South Africa
-0.4220563
Guinea-Bissau
0.0
Kenya
-0.04290999
Slovakia
-0.33804297
Myanmar
-0.4252847
Honduras
-0.05603922
Nicaragua
-0.010549765
Laos
-0.5154352
Croatia
-0.30469182
Belarus
-0.3309698
Netherlands
-0.6116485
Kazakhstan
-0.26559198
Bosnia and Herzegovina
-0.10748237
Tanzania
-0.045169134
Italy
-0.45113698
Thailand
-0.16381656
Ethiopia
-0.21410877
Costa Rica
-0.27426535
Canada
-0.4858136
Germany
-0.5045182
Indonesia
-0.32243893
Philippines
-0.13481711
Tajikistan
-0.63405085
Switzerland
-0.52874625
Peru
-0.24581076
Somalia
-0.054801896
Turkmenist